In [94]:
import nltk
import string
import scipy
import numpy as np
from scipy import spatial
from nltk.corpus import brown
from gensim.models import KeyedVectors
from tqdm import tqdm

In [95]:
# extract most frequent 5000 words in brown corpus
all_texts = []
for cat in brown.categories():
    all_texts += brown.words(categories=cat)
fdist = nltk.FreqDist([w.lower() for w in all_texts])
W = [k for k, v in sorted(fdist.items(), key=lambda item: item[1], reverse=True)]
W = [w for w in W if w not in string.punctuation][:5000]

In [96]:
# human-evaluated words in rg paper
W_rg = ['cord', 'smile', 'rooster', 'voyage', 'noon', 'string', 'fruit', 'furnace', 'autograph', 'shore', 'automobile', 'wizard', 
'mound', 'stove', 'grin', 'implement', 'asylum', 'fruit', 'asylum', 'monk', 'graveyard', 'madhouse', 'glass', 'magician', 'boy', 
'rooster', 'cushion', 'jewel', 'monk', 'slave', 'asylum', 'cemetary', 'coast', 'forest', 'grin', 'lad', 'shore', 'woodland', 'monk', 
'oracle', 'boy', 'sage', 'automobile', 'cushion', 'mound', 'shore', 'lad', 'wizard', 'forest', 'graveyard', 'food', 'rooster', 
'cemetery', 'woodland', 'shore', 'voyage', 'bird', 'woodland', 'coast', 'hill', 'furnace', 'implement', 'crane', 'rooster', 
'hill', 'woodland', 'car', 'journey', 'cemetery', 'mound', 'glass', 'jewel', 'magician', 'oracle', 'crane', 'implement', 'brother', 'lad', 
'sage', 'wizard', 'oracle', 'sage', 'bird', 'crane', 'bird', 'cock', 'food', 'fruit', 'brother', 'monk', 'asylum', 'madhouse', 'furnace', 'stove', 
'magician', 'wizard', 'hill', 'mound', 'cord', 'string', 'glass', 'tumbler', 'grin', 'smile', 'serf', 'slave', 'journey', 'voyage',
'autograph', 'signature', 'coast', 'shore', 'forest', 'woodland', 'implement', 'tool', 'cock', 'rooster', 'boy', 'lad', 'cushion', 'pillow', 'cemetery', 
'graveyard', 'automobile', 'car', 'midday', 'noon', 'gem', 'jewel']

W_rg_list = list(set(W_rg))
W = W + W_rg_list
W2idx = {W[i]:i for i in range(len(W))}

# human similarity scores
human_similarities = [0.02, 0.04, 0.04, 0.05, 0.06, 0.11, 0.14, 0.18, 0.19, 0.39, 0.42, 0.44, 0.44,
                     0.45, 0.57, 0.79, 0.85, 0.88, 0.90, 0.91, 0.96, 0.97, 0.97, 0.99, 1.00, 1.09, 1.18,
                     1.22, 1.24, 1.26, 1.37, 1.41, 1.48, 1.55, 1.69, 1.78, 1.82, 2.37, 2.41, 2.46, 2.61, 
                     2.63, 2.63, 2.69, 2.74, 3.04, 3.11, 3.21, 3.29, 3.41, 3.45, 3.46, 3.46, 3.58, 
                     3.59, 3.60, 3.65, 3.66, 3.68, 3.82, 3.84, 3.88, 3.92, 3.94, 3.94]

In [5]:
# load word2vec model
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [97]:
# compute cosine similarities
cosine_sims = []
for i in range(65):
    w1 = W_rg[i]
    w2 = W_rg[i+1]
    e1 = word2vec_model[w1]
    e2 = word2vec_model[w2]
    # cosine distance
    cos_dis = spatial.distance.cosine(e1, e2)
    cosine_sim = 1 - cos_dis
    cosine_sims.append(cosine_sim)
       

In [98]:
# pearson correlation between word2vec and human similarities
scipy.stats.pearsonr(human_similarities, cosine_sims)

(0.3515880627865447, 0.0040801619046865615)

In [101]:
# M1: word-context model
m1 = nltk.ConditionalFreqDist(nltk.bigrams(brown.words()))
mat1 = []
for word in W:
    mat1_i = [0]*len(W)
    contexts = m1[word]
    for k in contexts.keys():
        k = k.lower()
        if k in W2idx:
            mat1_i[W2idx[k]] = contexts[k]
    mat1.append(mat1_i)
mat1 = np.nan_to_num(np.array(mat1))   

In [104]:
# M1+: compute ppmi
import math
N = len(all_texts)
p_ws = {}
ppmi_mat1 = []
for i in tqdm(range(len(W))):
    w = W[i]
    p_w = float(fdist[w])/N
    p_ws[w] = p_w
    
for i in tqdm(range(len(W))):
    w = W[i]
    p_w = p_ws[w]
    ppmi_row = [0]*len(W)
    for j in range(len(mat1[i])):
        c = W[j]
        p_c = p_ws[c]
        p_wc = float(mat1[i][j]) / N
        if p_w > 0 and p_c > 0:
            x = p_wc / (p_w * p_c)
            if x > 0:
                ppmi_wc = max([0, math.log(x)])
                ppmi_row[j] = ppmi_wc
    ppmi_mat1.append(ppmi_row)
ppmi_mat1 = np.nan_to_num(np.array(ppmi_mat1))           


100%|██████████| 5049/5049 [00:00<00:00, 263615.71it/s]

 32%|███▏      | 1638/5049 [00:16<00:28, 118.01it/s]


 64%|██████▍   | 3224/5049 [00:34<00:23, 78.48it/s]


 97%|█████████▋| 4922/5049 [00:51<00:01, 96.41it/s]


100%|██████████| 5049/5049 [00:53<00:00, 94.92it/s] 


In [105]:
ppmi_mat1

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.80912147],
       [1.47858696, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20584242, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 2.30869401, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [106]:
# M2_10, M2_100, M2_300: PCA on M1+
import numpy as np
from sklearn.decomposition import PCA
pca_10 = PCA(n_components=10)
pca_100 = PCA(n_components=100)
pca_300 = PCA(n_components=300)
m2_10 = pca_10.fit_transform(ppmi_mat1)
m2_100 = pca_100.fit_transform(ppmi_mat1)
m2_300 = pca_300.fit_transform(ppmi_mat1)



In [107]:
# word2vec for all models
m1_dict = {W[i]:mat1[i] for i in range(len(W))}
m1plus_dict = {W[i]:ppmi_mat1[i] for i in range(len(W))}
m2_10_dict = {W[i]:m2_10[i] for i in range(len(W))}
m2_100_dict = {W[i]:m2_100[i] for i in range(len(W))}
m2_300_dict = {W[i]:m2_300[i] for i in range(len(W))}

In [108]:
def person_cor(model):
    cosine_sims = []
    for i in range(65):
        w1 = W_rg[i]
        w2 = W_rg[i+1]
        e1 = model[w1]
        e2 = model[w2]
        # cosine distance
        cos_dis = spatial.distance.cosine(e1, e2)
        cosine_sim = 1 - cos_dis
        cosine_sims.append(cosine_sim)
    cosine_sims = np.array(cosine_sims)
    valid_idx = [i for i in range(65) if np.isfinite(cosine_sims[i])]
    valid_human_sims = [human_similarities[i] for i in valid_idx]
    valid_cosine_sims = [cosine_sims[i] for i in valid_idx]
    print("number of valid examples: ", len(valid_idx))
    return scipy.stats.pearsonr(valid_human_sims, valid_cosine_sims)

In [109]:
for model in [m1_dict, m1plus_dict, m2_10_dict, m2_100_dict, m2_300_dict]:
    print(person_cor(model))

number of valid examples:  39
(-0.0035371783298064202, 0.9829496963412535)
number of valid examples:  39
(0.20316282560178833, 0.2148062013536032)
number of valid examples:  65
(0.036183049039045304, 0.7747628437471944)
number of valid examples:  65
(0.15382391960802208, 0.2211780570705272)
number of valid examples:  65
(0.18519059188017264, 0.13970448695077833)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [110]:
# from sklearn.neighbors import NearestNeighbors
# nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X)

# perform analogy test
f_syn = open("analogy_syn.txt", "r")
syn_lines = f_syn.readlines()

f_sem = open("analogy_sem.txt", "r")
sem_lines = f_sem.readlines()

def analogy_test(model):
    syn_acc = 0
    sem_acc = 0
    
    valid_syn_idx = []
    valid_sem_idx = []
    vec2w = {}
    for w in W:
        if w in model:
            vec2w[w] = model[w]
    for i in tqdm(range(len(syn_lines))):
        syn_line = syn_lines[i]
        ws = syn_line.split()
        ground_truth = ws[3].lower()
        if ws[0] in model and ws[1] in model and ws[2] in model:
            valid_syn_idx.append(i)
            pred_word = model.most_similar(positive=[ws[1], ws[2]], negative=[ws[0]], topn=1)[0][0].lower()
            if pred_word == ground_truth:
                syn_acc += 1
    for i in tqdm(range(len(sem_lines))):
        sem_line = sem_lines[i]
        ws = sem_line.split()
        ground_truth = ws[3].lower()
        if ws[0] in model and ws[1] in model and ws[2] in model:
            valid_sem_idx.append(i)
            pred_word = model.most_similar(positive=[ws[1], ws[2]], negative=[ws[0]], topn=1)[0][0].lower()
            if pred_word == ground_truth:
                sem_acc += 1
    print("valid syn examples: ", len(valid_syn_idx))
    print("valid sem examples: ", len(valid_sem_idx))
    return float(syn_acc) / len(valid_syn_idx), float(sem_acc) / len(valid_sem_idx)
    

In [71]:
# m_w2v = model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
# models = [m1_dict, m1plus_dict, m2_10_dict, m2_100_dict, m2_300_dict, m_w2v]
# for model in models:
#     syn_acc, sem_acc = analogy_test(model)
#     print('syn_acc: ' + syn_acc + 'sem_acc: ' + sem_acc)
syn_acc, sem_acc = analogy_test(word2vec_model)
print(syn_acc)
print(sem_acc)




  0%|          | 0/10675 [00:00<?, ?it/s]


  0%|          | 1/10675 [00:02<5:56:06,  2.00s/it]


  0%|          | 2/10675 [00:02<4:29:12,  1.51s/it]


  0%|          | 3/10675 [00:02<3:27:47,  1.17s/it]


  0%|          | 4/10675 [00:03<2:43:39,  1.09it/s]


  0%|          | 5/10675 [00:03<2:14:42,  1.32it/s]


  0%|          | 6/10675 [00:03<1:55:19,  1.54it/s]


  0%|          | 7/10675 [00:04<1:39:18,  1.79it/s]


  0%|          | 8/10675 [00:04<1:27:53,  2.02it/s]


  0%|          | 9/10675 [00:04<1:17:24,  2.30it/s]


  0%|          | 10/10675 [00:05<1:07:50,  2.62it/s]


  0%|          | 11/10675 [00:05<1:04:47,  2.74it/s]


  0%|          | 12/10675 [00:05<59:49,  2.97it/s]  


  0%|          | 13/10675 [00:05<57:22,  3.10it/s]


  0%|          | 14/10675 [00:06<53:15,  3.34it/s]


  0%|          | 15/10675 [00:06<50:40,  3.51it/s]


  0%|          | 16/10675 [00:06<49:13,  3.61it/s]


  0%|          | 17/10675 [00:06<47:49,  3.71it/s]


  0%|          | 18/10675 [00:07<46:5

  2%|▏         | 261/10675 [01:07<41:48,  4.15it/s]


  2%|▏         | 262/10675 [01:07<42:24,  4.09it/s]


  2%|▏         | 263/10675 [01:07<42:14,  4.11it/s]


  2%|▏         | 264/10675 [01:08<41:36,  4.17it/s]


  2%|▏         | 265/10675 [01:08<41:12,  4.21it/s]


  2%|▏         | 266/10675 [01:08<40:27,  4.29it/s]


  3%|▎         | 267/10675 [01:08<40:02,  4.33it/s]


  3%|▎         | 268/10675 [01:09<39:51,  4.35it/s]


  3%|▎         | 269/10675 [01:09<40:09,  4.32it/s]


  3%|▎         | 270/10675 [01:09<39:55,  4.34it/s]


  3%|▎         | 271/10675 [01:09<40:18,  4.30it/s]


  3%|▎         | 272/10675 [01:10<41:00,  4.23it/s]


  3%|▎         | 273/10675 [01:10<40:09,  4.32it/s]


  3%|▎         | 274/10675 [01:10<40:08,  4.32it/s]


  3%|▎         | 275/10675 [01:10<40:35,  4.27it/s]


  3%|▎         | 276/10675 [01:10<40:55,  4.23it/s]


  3%|▎         | 277/10675 [01:11<41:10,  4.21it/s]


  3%|▎         | 278/10675 [01:11<41:48,  4.14it/s]


  3%|▎         | 279/10675 [

  5%|▍         | 521/10675 [02:10<42:33,  3.98it/s]


  5%|▍         | 522/10675 [02:11<41:43,  4.06it/s]


  5%|▍         | 523/10675 [02:11<41:06,  4.12it/s]


  5%|▍         | 524/10675 [02:11<41:14,  4.10it/s]


  5%|▍         | 525/10675 [02:11<40:46,  4.15it/s]


  5%|▍         | 526/10675 [02:12<40:08,  4.21it/s]


  5%|▍         | 527/10675 [02:12<40:00,  4.23it/s]


  5%|▍         | 528/10675 [02:12<40:08,  4.21it/s]


  5%|▍         | 529/10675 [02:12<40:49,  4.14it/s]


  5%|▍         | 530/10675 [02:13<40:44,  4.15it/s]


  5%|▍         | 531/10675 [02:13<40:42,  4.15it/s]


  5%|▍         | 532/10675 [02:13<40:34,  4.17it/s]


  5%|▍         | 533/10675 [02:13<41:06,  4.11it/s]


  5%|▌         | 534/10675 [02:14<40:31,  4.17it/s]


  5%|▌         | 535/10675 [02:14<40:54,  4.13it/s]


  5%|▌         | 536/10675 [02:14<40:56,  4.13it/s]


  5%|▌         | 537/10675 [02:14<40:49,  4.14it/s]


  5%|▌         | 538/10675 [02:14<40:32,  4.17it/s]


  5%|▌         | 539/10675 [

  7%|▋         | 781/10675 [03:14<41:57,  3.93it/s]


  7%|▋         | 782/10675 [03:14<41:54,  3.94it/s]


  7%|▋         | 783/10675 [03:15<40:49,  4.04it/s]


  7%|▋         | 784/10675 [03:15<40:37,  4.06it/s]


  7%|▋         | 785/10675 [03:15<39:51,  4.14it/s]


  7%|▋         | 786/10675 [03:15<39:25,  4.18it/s]


  7%|▋         | 787/10675 [03:16<38:30,  4.28it/s]


  7%|▋         | 788/10675 [03:16<38:49,  4.24it/s]


  7%|▋         | 789/10675 [03:16<43:05,  3.82it/s]


  7%|▋         | 790/10675 [03:17<49:50,  3.30it/s]


  7%|▋         | 791/10675 [03:17<49:15,  3.34it/s]


  7%|▋         | 792/10675 [03:17<47:24,  3.47it/s]


  7%|▋         | 793/10675 [03:17<46:08,  3.57it/s]


  7%|▋         | 794/10675 [03:18<44:42,  3.68it/s]


  7%|▋         | 795/10675 [03:18<43:28,  3.79it/s]


  7%|▋         | 796/10675 [03:18<42:30,  3.87it/s]


  7%|▋         | 797/10675 [03:18<41:40,  3.95it/s]


  7%|▋         | 798/10675 [03:19<41:13,  3.99it/s]


  7%|▋         | 799/10675 [

 10%|▉         | 1039/10675 [04:23<40:43,  3.94it/s]


 10%|▉         | 1040/10675 [04:23<40:25,  3.97it/s]


 10%|▉         | 1041/10675 [04:23<40:01,  4.01it/s]


 10%|▉         | 1042/10675 [04:23<40:02,  4.01it/s]


 10%|▉         | 1043/10675 [04:24<39:36,  4.05it/s]


 10%|▉         | 1044/10675 [04:24<39:27,  4.07it/s]


 10%|▉         | 1045/10675 [04:24<39:39,  4.05it/s]


 10%|▉         | 1046/10675 [04:24<39:39,  4.05it/s]


 10%|▉         | 1047/10675 [04:25<39:27,  4.07it/s]


 10%|▉         | 1048/10675 [04:25<39:35,  4.05it/s]


 10%|▉         | 1049/10675 [04:25<39:17,  4.08it/s]


 10%|▉         | 1050/10675 [04:25<39:23,  4.07it/s]


 10%|▉         | 1051/10675 [04:26<39:06,  4.10it/s]


 10%|▉         | 1052/10675 [04:26<40:20,  3.97it/s]


 10%|▉         | 1053/10675 [04:26<40:11,  3.99it/s]


 10%|▉         | 1054/10675 [04:26<39:45,  4.03it/s]


 10%|▉         | 1055/10675 [04:27<39:45,  4.03it/s]


 10%|▉         | 1056/10675 [04:27<40:00,  4.01it/s]


 10%|▉    

...

 89%|████████▉ | 7896/8869 [31:18<03:46,  4.30it/s]


 89%|████████▉ | 7897/8869 [31:19<03:54,  4.15it/s]


 89%|████████▉ | 7898/8869 [31:19<03:53,  4.17it/s]


 89%|████████▉ | 7899/8869 [31:19<03:53,  4.16it/s]


 89%|████████▉ | 7900/8869 [31:19<03:51,  4.18it/s]


 89%|████████▉ | 7901/8869 [31:20<03:48,  4.24it/s]


 89%|████████▉ | 7902/8869 [31:20<03:49,  4.21it/s]


 89%|████████▉ | 7903/8869 [31:20<03:46,  4.26it/s]


 89%|████████▉ | 7904/8869 [31:20<03:52,  4.16it/s]


 89%|████████▉ | 7905/8869 [31:21<03:56,  4.08it/s]


 89%|████████▉ | 7906/8869 [31:21<03:54,  4.10it/s]


 89%|████████▉ | 7907/8869 [31:21<03:53,  4.12it/s]


 89%|████████▉ | 7908/8869 [31:21<03:50,  4.17it/s]


 89%|████████▉ | 7909/8869 [31:22<03:49,  4.18it/s]


 89%|████████▉ | 7910/8869 [31:22<03:48,  4.19it/s]


 89%|████████▉ | 7911/8869 [31:22<03:47,  4.22it/s]


 89%|████████▉ | 7912/8869 [31:22<03:49,  4.18it/s]


 89%|████████▉ | 7913/8869 [31:22<03:53,  4.10it/s]


 89%|████████▉ | 7914/8869 [

 92%|█████████▏| 8156/8869 [32:20<02:49,  4.22it/s]


 92%|█████████▏| 8157/8869 [32:20<02:46,  4.27it/s]


 92%|█████████▏| 8158/8869 [32:20<02:45,  4.30it/s]


 92%|█████████▏| 8159/8869 [32:20<02:45,  4.28it/s]


 92%|█████████▏| 8160/8869 [32:20<02:53,  4.09it/s]


 92%|█████████▏| 8161/8869 [32:21<02:46,  4.26it/s]


 92%|█████████▏| 8162/8869 [32:21<02:43,  4.32it/s]


 92%|█████████▏| 8163/8869 [32:21<02:43,  4.33it/s]


 92%|█████████▏| 8164/8869 [32:21<02:44,  4.28it/s]


 92%|█████████▏| 8165/8869 [32:22<02:44,  4.28it/s]


 92%|█████████▏| 8166/8869 [32:22<02:42,  4.32it/s]


 92%|█████████▏| 8167/8869 [32:22<02:44,  4.27it/s]


 92%|█████████▏| 8168/8869 [32:22<02:43,  4.28it/s]


 92%|█████████▏| 8169/8869 [32:23<02:40,  4.36it/s]


 92%|█████████▏| 8170/8869 [32:23<02:38,  4.42it/s]


 92%|█████████▏| 8171/8869 [32:23<02:36,  4.45it/s]


 92%|█████████▏| 8172/8869 [32:23<02:38,  4.39it/s]


 92%|█████████▏| 8173/8869 [32:23<02:38,  4.40it/s]


 92%|█████████▏| 8174/8869 [

 95%|█████████▍| 8416/8869 [33:21<01:49,  4.15it/s]


 95%|█████████▍| 8417/8869 [33:21<01:47,  4.21it/s]


 95%|█████████▍| 8418/8869 [33:21<01:46,  4.24it/s]


 95%|█████████▍| 8419/8869 [33:22<01:45,  4.25it/s]


 95%|█████████▍| 8420/8869 [33:22<01:46,  4.21it/s]


 95%|█████████▍| 8421/8869 [33:22<01:46,  4.22it/s]


 95%|█████████▍| 8422/8869 [33:22<01:45,  4.24it/s]


 95%|█████████▍| 8423/8869 [33:23<01:42,  4.35it/s]


 95%|█████████▍| 8424/8869 [33:23<01:43,  4.31it/s]


 95%|█████████▍| 8425/8869 [33:23<01:44,  4.27it/s]


 95%|█████████▌| 8426/8869 [33:23<01:42,  4.33it/s]


 95%|█████████▌| 8427/8869 [33:24<01:42,  4.30it/s]


 95%|█████████▌| 8428/8869 [33:24<01:42,  4.32it/s]


 95%|█████████▌| 8429/8869 [33:24<01:43,  4.25it/s]


 95%|█████████▌| 8430/8869 [33:24<01:43,  4.24it/s]


 95%|█████████▌| 8431/8869 [33:24<01:41,  4.32it/s]


 95%|█████████▌| 8432/8869 [33:25<01:39,  4.39it/s]


 95%|█████████▌| 8433/8869 [33:25<01:41,  4.30it/s]


 95%|█████████▌| 8434/8869 [

 98%|█████████▊| 8676/8869 [34:23<00:45,  4.27it/s]


 98%|█████████▊| 8677/8869 [34:23<00:44,  4.34it/s]


 98%|█████████▊| 8678/8869 [34:23<00:44,  4.29it/s]


 98%|█████████▊| 8679/8869 [34:23<00:44,  4.29it/s]


 98%|█████████▊| 8680/8869 [34:24<00:43,  4.33it/s]


 98%|█████████▊| 8681/8869 [34:24<00:43,  4.31it/s]


 98%|█████████▊| 8682/8869 [34:24<00:43,  4.28it/s]


 98%|█████████▊| 8683/8869 [34:24<00:42,  4.37it/s]


 98%|█████████▊| 8684/8869 [34:25<00:42,  4.40it/s]


 98%|█████████▊| 8685/8869 [34:25<00:41,  4.40it/s]


 98%|█████████▊| 8686/8869 [34:25<00:41,  4.38it/s]


 98%|█████████▊| 8687/8869 [34:25<00:40,  4.44it/s]


 98%|█████████▊| 8688/8869 [34:26<00:40,  4.43it/s]


 98%|█████████▊| 8689/8869 [34:26<00:40,  4.42it/s]


 98%|█████████▊| 8690/8869 [34:26<00:40,  4.44it/s]


 98%|█████████▊| 8691/8869 [34:26<00:40,  4.45it/s]


 98%|█████████▊| 8692/8869 [34:26<00:40,  4.40it/s]


 98%|█████████▊| 8693/8869 [34:27<00:40,  4.35it/s]


 98%|█████████▊| 8694/8869 [

0.7401405152224825
0.7308602999210734


In [55]:
# write lsa 300-d embeddings into file
with open('lsa_300d.txt', 'w') as lsa_file:
    lsa_file.write('5049 300\n')
    n = m2_300.shape[0]
    for i in range(n):
        w = W[i]
        weights = [str(x) for x in m2_300[i]]
        lsa_file.write(' '.join([w] + weights) + '\n')

In [56]:
lsa_300_model = KeyedVectors.load_word2vec_format(open('lsa_300d.txt'), binary=False)

In [111]:
syn_acc_lsa, sem_acc_lsa = analogy_test(lsa_300_model)
print(syn_acc_lsa)
print(sem_acc_lsa)


100%|██████████| 10675/10675 [00:04<00:00, 2624.17it/s]

100%|██████████| 8869/8869 [00:00<00:00, 51112.31it/s]

valid syn examples:  3094
valid sem examples:  120
0.062055591467356175
0.15


In [112]:
len(syn_lines)

10675

In [113]:
len(sem_lines)

8869

In [115]:
# import diachronic word embeddings
import pickle
diach_embeddings = pickle.load(open('/Users/leiyu/Downloads/data/data.pkl', 'rb'))   

In [170]:
vocab = diach_embeddings['w']
decades = diach_embeddings['d']
E = diach_embeddings['E']

In [129]:
arr = np.array(diach_embeddings['E'])

In [133]:
# extract embeddings from each decade into matrices
diach_mats = arr.reshape(10, 2000, -1)

In [134]:
# Procrustes aligning method
def procrustes_align(base_embed, other_embed):
    """ 
        Align other embedding to base embeddings via Procrustes.
        Returns best distance-preserving aligned version of other_embed
        NOTE: Assumes indices are aligned
    """
    basevecs = base_embed - base_embed.mean(0)
    othervecs = other_embed - other_embed.mean(0)
    m = othervecs.T.dot(basevecs)
    u, _, v = np.linalg.svd(m)
    ortho = u.dot(v)
    fixedvecs = othervecs.dot(ortho)
    return fixedvecs


In [136]:
base_embed = diach_mats[0]
other_embed = diach_mats[1]

fixed_other_embed = procrustes_align(base_embed, other_embed)

In [137]:
embed_diff = fixed_other_embed - base_embed

In [138]:
embed_diff.shape

(2000, 300)

In [238]:
# align embeddings via Procrustes method, and compute differences between each decade
embed_diffs = []
for i in range(9):
    base_embed = diach_mats[i]
    other_embed = diach_mats[i+1]
    
    fixed_other_embed = procrustes_align(base_embed, other_embed)
    embed_diff = fixed_other_embed - base_embed
    diff_norms = np.linalg.norm(embed_diff, axis=1)
    
    embed_diffs.append(diff_norms)
    
    
embed_diffs = np.stack(embed_diffs, axis=0)        

In [239]:
embed_diffs.shape

(9, 2000)

In [240]:
# compute mean variations (Euclidean distance)
mean_variations = embed_diffs.mean(0)

In [241]:
max_change_idx = mean_variations.argsort()[-20:][::-1]
min_change_idx = mean_variations.argsort()[:20][::-1]

In [242]:
max_change_words = [vocab[i] for i in list(max_change_idx)]
min_change_words = [vocab[i] for i in list(min_change_idx)]

In [243]:
', '.join(min_change_words)

'publications, motor, copy, measurements, minute, fleet, views, delay, supreme, rome, feelings, papers, wave, stranger, leaves, requirements, controversy, taxation, issues, conflict'

In [244]:
', '.join(max_change_words)

'error, fingers, mission, project, place, thirds, thousands, wilson, medium, records, york, reference, letter, ship, wages, ears, problems, scotland, nothing, tale'

In [175]:
base_embed = diach_mats[0]
other_embed = diach_mats[-1]

fixed_other_embed = procrustes_align(base_embed, other_embed)
embed_diff = fixed_other_embed - base_embed
diff_norms = np.linalg.norm(embed_diff, axis=1)

max_change_idx = diff_norms.argsort()[-20:][::-1]
min_change_idx = diff_norms.argsort()[:20][::-1]

In [176]:
max_change_words = [vocab[i] for i in list(max_change_idx)]
min_change_words = [vocab[i] for i in list(min_change_idx)]

In [177]:
max_change_words

['lords',
 'depth',
 'film',
 'observations',
 'parent',
 'sky',
 'emphasis',
 'columbia',
 'aspect',
 'plenty',
 'stages',
 'moon',
 'levels',
 'decades',
 'identity',
 'crown',
 'story',
 'frame',
 'grand',
 'offices']

In [178]:
min_change_words

['diet',
 'deposits',
 'act',
 'centre',
 'regret',
 'thickness',
 'ice',
 'souls',
 'side',
 'achievement',
 'bulk',
 'unions',
 'sample',
 'facilities',
 'safety',
 'shock',
 'columns',
 'transfer',
 'assumption',
 'vessels']

In [226]:
# knn method computing semnatic variations
from sklearn.neighbors import NearestNeighbors
neighbors = []
for i in tqdm(range(10)):
    embeddings = diach_mats[i]
    nbrs = NearestNeighbors(n_neighbors=100, algorithm='auto').fit(embeddings)
    _, nns = nbrs.kneighbors(embeddings)
    neighbors.append(nns)
    
    


100%|██████████| 10/10 [00:24<00:00,  2.41s/it]


In [227]:
# find words with most neighbors changed
diff_nns = []
for i in range(2000):
    diff_nns_w = 0
    for j in range(9):
        diff_nns_w += sum(neighbors[j+1][i] == neighbors[j][i])
    diff_nns_w = float(diff_nns_w) / 9
    diff_nns.append(diff_nns_w)
diff_nns = np.array(diff_nns)        

In [228]:
max_change_idx = diff_nns.argsort()[-20:][::-1]
min_change_idx = diff_nns.argsort()[:20][::-1]

In [229]:
max_change_words = [vocab[i] for i in list(max_change_idx)]
min_change_words = [vocab[i] for i in list(min_change_idx)]

In [231]:
print(', '.join(max_change_words))
print(', '.join(min_change_words))

train, technology, benefits, census, firms, silence, negotiations, channel, carolina, response, habit, ranks, characters, host, faith, meat, percent, colonies, alcohol, harper
health, approval, stranger, weapons, wiley, streets, executive, bureau, villages, wine, finance, child, instant, manuscript, sets, method, gate, tendency, qualities, alliance


In [254]:
from scipy.spatial.distance import cosine
embed_diffs = []
for i in range(9):
    base_embed = diach_mats[0]
    other_embed = diach_mats[-1]
    
    fixed_other_embed = procrustes_align(base_embed, other_embed)
    cosine_dists = []
    for j in range(2000):
        cosine_dists.append(cosine(fixed_other_embed[j], base_embed[j]))
    
    embed_diffs.append(np.array(cosine_dists))
    
    
embed_diffs = np.stack(embed_diffs, axis=0)        

In [255]:
mean_variations = embed_diffs.mean(0)

max_change_idx = mean_variations.argsort()[-20:][::-1]
min_change_idx = mean_variations.argsort()[:20][::-1]

max_change_words = [vocab[i] for i in list(max_change_idx)]
min_change_words = [vocab[i] for i in list(min_change_idx)]

In [258]:
' ,'.join(min_change_words)

'neck ,leader ,regret ,thickness ,side ,souls ,jerusalem ,centre ,ice ,bulk ,achievement ,unions ,facilities ,sample ,shock ,safety ,columns ,assumption ,transfer ,vessels'

In [257]:
', '.join(max_change_words)

'lords, film, depth, observations, parent, emphasis, sky, aspect, plenty, columbia, levels, identity, frame, stages, story, offices, moon, decades, berlin, roads'